In [289]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
df = pd.read_csv('NPPE1_Preprocessing1.csv')
df.head()

,CRIM,ZN,INDUS,POLINDEX,RM,AGE,DIS,HIGHWAYCOUNT,TAX,PTRATIO,IMM,BPL,PRICE,RIVERSIDE
0,1.026769,1.429034,7.851300,1.134216,6.0,42.0,5.251911,5,279.201277,20.689586,398.811960,10.461456,22.991633,NO
1,0.848089,0.255543,6.263434,1.245993,7.0,63.0,4.305546,8,307.444529,17.465398,377.153649,11.619690,24.551055,NO
2,10.925905,0.441022,18.322960,2.824833,8.0,-2.0,2.409495,25,666.492973,20.351601,387.061355,19.366070,15.875346,NO
3,0.559027,1.041175,11.114920,0.794952,6.0,9.0,6.898669,4,305.514181,19.787314,391.778647,6.206820,23.007756,NO
4,0.905063,81.167963,3.673369,1.029030,8.0,20.0,10.246463,1,315.913960,17.360439,395.833166,10.827105,21.503177,NO


How many samples are there in the dataset?

In [290]:
df.shape[0]

4000

What is the average house price (in lacs)?

In [291]:
df['PRICE'].mean()

24.355923220694248

How many houses have 5 or more rooms?

In [292]:
(df["RM"] >= 5).sum()

3953

What is the average price of the top 10 most expensive houses (in lacs)?

In [293]:
df['PRICE'].sort_values(ascending = False).head(10).mean()

52.36590175716407


What is the total number of missing or unknown values in the number of rooms feature?

In [294]:
df[ (df['RM'] <= 0)].shape[0]

40

What is the total number of missing or unknown values in the age feature?


In [295]:
age_miss = df['AGE'].isnull().sum() + df[(df['AGE'] < 0) | (df['AGE'] > 200)].shape[0]
age_miss

50

What is the total number of missing or unknown values in the RIVERSIDE feature?

In [296]:
river_miss = df['RIVERSIDE'].isnull().sum() + df[~df['RIVERSIDE'].isin(['NO', 'YES'])].shape[0]
river_miss

88

How many houses are on riverside and were built within the last 50 years (i.e. a house 50 years old or younger)? For this question, ignore the rows that have missing values in either riverside feature or age feature.

In [297]:
df[(df['RIVERSIDE']=='YES')& (df['AGE'] > 0)&(df['AGE']<=50)].shape[0]

44

How many houses are near to exactly 6, 7 or 8 highways (all three inclusive)?

In [298]:
df['HIGHWAYCOUNT'].isin([6,7,8]).sum()

1211


Create a column 'CATEGORY' and divide the houses in categories as following:

Category 1: house price <10 lacs

Category 2: 10 lacs <= house price <20 lacs

Category 3: 20 lacs <= house price <30 lacs

Category 4: 30 lacs <= house price <40 lacs

Category 5: house price >=40 lacs

Which category has the highest number of records?

In [299]:
category_labels = ['Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5']
df['CATEGORY']= pd.cut(df['PRICE'], bins= [0, 10, 20, 30, 40, float('inf')], labels=category_labels, right=False)
df['CATEGORY'].value_counts().idxmax()

'Category 3'

Apply Pre processing
Divide the data into training and test sets

1. Replace the respective missing or unknown values in features room count, riverside and age with np.nan.
2. Keep 30% of the data as test set.
3. Use random_state as 0
4. PRICE is the target, rest of the columns are the features.
5. Apply train test split.


In [300]:
# Replace the respective missing or unknown values in features room count, riverside and age with np.nan.
import numpy as np
df['RM']=df['RM'].apply(lambda x: np.nan if x<=0 else x)
df['RIVERSIDE'] = df['RIVERSIDE'].apply(lambda x: np.nan if x not in ['YES', 'NO'] else x)
df['AGE'] = df['AGE'].apply(lambda x: np.nan if x>200 or x <= 0 else x)

# Keep 30% of the data as test set and random_state as 0
from sklearn.model_selection import train_test_split
X = df.drop(columns =['PRICE'])
Y = df['PRICE']

X_train, X_test ,Y_train,Y_test = train_test_split(X, Y, test_size =0.30 ,random_state=0)



What is the number of samples in the training set?

In [301]:
X_train.shape[0]

2800

Apply following preprocessing steps:

1. Drop CATEGORY column
2. CRIM: min max scaling
3. ZN: min max scaling
4. INDUS: standard scaling
5. POLINDEX: min max scaling
6. DIS: min max scaling
7. HIGHWAYCOUNT: min max scaling
8. TAX: min max scaling
9. PTRATIO: min max scaling
10. IMM: min max scaling
11. BPL: min max scaling
12. RM: impute with median then min max scaling
13. AGE: impute with mean then min max scaling
14. RIVERSIDE: Impute with most frequent value then one hot encode.


In [302]:
df = df.drop(columns='CATEGORY')
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
min_max = MinMaxScaler()
scal = StandardScaler()

imp = SimpleImputer(strategy = 'median')
imp_mean = SimpleImputer(strategy = 'mean')
imp_freq = SimpleImputer(strategy='most_frequent')

X_train[['RM']] = imp.fit_transform(X_train[['RM']])
X_test[['RM']] = imp.fit_transform(X_test[['RM']])

X_train[['RM']] = min_max.fit_transform(X_train[['RM']])
X_test[['RM']] = min_max.fit_transform(X_test[['RM']])


min_max_cols = ['CRIM', 'ZN', 'POLINDEX', 'DIS', 'HIGHWAYCOUNT', 'TAX', 'PTRATIO', 'IMM', 'BPL','RM','AGE']
standard_cols = ['INDUS']
riverside_col = ['RIVERSIDE']

X_train[min_max_cols] = min_max.fit_transform(X_train[min_max_cols])
X_test[min_max_cols] = min_max.fit_transform(X_test[min_max_cols])

X_train[standard_cols] = scal.fit_transform(X_train[standard_cols])
X_test[standard_cols] = scal.fit_transform(X_test[standard_cols])

X_train[riverside_col] = imp_freq.fit_transform(X_train[riverside_col])
X_test[riverside_col] = imp_freq.fit_transform(X_test[riverside_col])

X_train = pd.get_dummies(X_train, columns=riverside_col, drop_first=True)
X_test = pd.get_dummies(X_test, columns=riverside_col, drop_first=True)





How many features are there after performing above transformation?

In [303]:
X_train.shape[1]

14

What is the mean of the transformed test data (features only)?
Note : Compute the mean of the whole feature matrix i.e. mean of all values in the transformed test feature matrix

In [304]:
X_test_numeric =X_test.select_dtypes(include=['number']).mean()
X_test_numeric.mean()

0.3662646710700444